In [55]:
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from numpy import asarray
from numpy import mean
from numpy import std
from xgboost import XGBRegressor
import xgboost

0.7821003944033458 => sequenceLength:10, nTrees: 64, max_depth: 50

0.7944474821071543 => sequenceLength:10, nTrees: 64, max_depth: None

0.7905605781030345 => sequenceLength:10, nTrees: 64, max_depth: None, labels MinMax scaled

0.7893843537936445 => sequenceLength:10, nTrees: 64, max_depth: None, labels standard scaled

0.8101881801103619 => sequenceLength:20, nTrees: 64, max_depth: None, labels standard scaled

### XGBoost (double model)
* 0.7740843741300616, 0.6670367841427146 => sequenceLength:20, nTrees: 64, max_depth: default, labels standard scaled
* 0.7339056408411957, 0.6153000034595998 => sequenceLength:20, nTrees: 64, max_depth: default, labels standard scaled, gpu_hist
* 0.8160022012880737, 0.7179601372210322 => sequenceLength:20, nTrees: 64, max_depth: 10, labels standard scaled, gpu_hist
* 0.84167863251325, 0.7641087580768761 => sequenceLength:20, nTrees: 64, max_depth: 20, labels standard scaled, gpu_hist

### RandomForest (double model)
* 0.83827540937785, 0.7663303903141425 => sequenceLength:20, nTrees: 32, max_depth: 50, labels standard scaled
* 0.8433744923454447,0.7731579198827838 => sequenceLength:20, nTrees: 64, max_depth: 50, labels standard scaled
* 0.8430462022005601, 0.7724669773855409 => sequenceLength:20, nTrees: 64, max_depth: 50, labels standard scaled, bootstrap=True
#### After Introducing trajectory() features
##### RF
* 0.9832796800593451, 0.9871431958518339 => sequenceLength:20, nTrees: 64, max_depth: 50, labels standard scaled, bootstrap=True
##### XG
* 0.9782034741965696, 0.9856036857164658 => sequenceLength:20, nTrees: 64, max_depth: 20, labels standard scaled, bootstrap=True
* 0.9801315191585513, 0.9866471744961971 => sequenceLength:20, nTrees: 64, max_depth: 20, labels standard scaled, bootstrap=True, learning_rate=0.2
* 0.9818559205025458, 0.9871650521809248 => sequenceLength:20, nTrees: 64, max_depth: 20, labels standard scaled, bootstrap=True, learning_rate=0.1

## Load Data

In [64]:
use_windows = True
if use_windows:
    dbfile = open('otherLargeFiles/CNN-dataset.pkl', 'rb')     
else:
    dbfile = open('otherLargeFiles/CNN-dataset-no-window.pkl', 'rb')     
dataset = pickle.load(dbfile)
dbfile.close()

dataset["samples"] = dataset["samples"][:,:,(0,1,2,5,7,9,10,11,12,13,14,15,16)]
if len(dataset["samples"].shape) == 3:
    dataset["samples"] = dataset["samples"].reshape(dataset["samples"].shape[0],dataset["samples"].shape[1]*dataset["samples"].shape[2])
    
    
testDriveFile = open('otherLargeFiles/test-drive-0.pkl', 'rb')
testDrive = pickle.load(testDriveFile)
testDriveFile.close()
testDrive["samples"] = testDrive["samples"][:,:,(0,1,2,5,7,9,10,11,12,13,14,15,16)]
if len(testDrive["samples"].shape) == 3:
    testDrive["samples"] = testDrive["samples"].reshape(testDrive["samples"].shape[0],testDrive["samples"].shape[1]*testDrive["samples"].shape[2])

## Split into training/validation

In [65]:
trainInputs, testInputs, trainLabels, testLabels = train_test_split(dataset["samples"], dataset["labels"], test_size=0.5, shuffle=True)

# The Model

In [66]:
trainInputs.shape

(272419, 130)

In [118]:
rfLongModel = RandomForestRegressor(n_estimators=64, max_depth=50, verbose=100, n_jobs=-1, bootstrap=True)
rfLatModel = RandomForestRegressor(n_estimators=64, max_depth=50, verbose=100, n_jobs=-1, bootstrap=True)

rfLongModel.fit(trainInputs, trainLabels[:,0])
rfLatModel.fit(trainInputs, trainLabels[:,1])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
building tree 1 of 64building tree 2 of 64
building tree 3 of 64
building tree 4 of 64
building tree 5 of 64
building tree 6 of 64

building tree 7 of 64
building tree 8 of 64
building tree 9 of 64[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.1min

building tree 10 of 64[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.1min

building tree 11 of 64[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.1min

building tree 12 of 64[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.1min

building tree 13 of 64[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.2min

building tree 14 of 64[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.2min

building tree 15 of 64[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.2min

building tree 16 of 64[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.2min

building tree 17 of 64[Parallel(n_jobs=-1)]: Done   9 tasks     

RandomForestRegressor(max_depth=50, n_estimators=64, n_jobs=-1, verbose=100)

In [67]:
xgbLongModel = XGBRegressor(eta=0.1, n_estimators=64, max_depth=20, verbosity=3, n_jobs=-1, objective='reg:squarederror', tree_method="gpu_hist")
xgbLatModel = XGBRegressor(eta=0.1, n_estimators=64, max_depth=20, verbosity=3, n_jobs=-1, objective='reg:squarederror', tree_method="gpu_hist")
xgbLongModel.fit(trainInputs, trainLabels[:,0])
xgbLatModel.fit(trainInputs, trainLabels[:,1])

[17:07:58] ======== Monitor: Learner ========
[17:07:58] Configure: 0.001353s, 1 calls @ 1353us

[17:07:58] ======== Monitor: GBTree ========
[17:07:58] ======== Device 0 Memory Allocations:  ========
[17:07:58] Peak memory usage: 1316MiB
[17:07:58] Number of allocations: 15018934
[17:07:58] ======== Monitor: updater_gpu_hist ========
[17:07:58] ======== NCCL Statistics========
[17:07:58] AllReduce calls: 0
[17:07:58] AllReduce total MiB communicated: 0


/home/isaac/anaconda3/lib/python3.8/site-packages/xgboost/data.py:119: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[17:07:58] DEBUG: ../src/tree/updater_gpu_hist.cu:899: [GPU Hist]: Configure
[17:07:58] DEBUG: ../src/common/device_helpers.cu:38: Running nccl init on: 10.0
[17:07:58] ======== Monitor: SketchContainer ========
[17:07:58] MakeCuts: 0.000282s, 1 calls @ 282us

[17:07:58] Prune: 0.000206s, 1 calls @ 206us

[17:07:58] ScanInput: 0.002492s, 1 calls @ 2492us

[17:07:58] Unique: 0.000163s, 1 calls @ 163us

[17:10:49] ======== Monitor: Learner ========
[17:10:49] Configure: 0.000511s, 1 calls @ 511us

[17:10:49] EvalOneIter: 0.000619s, 64 calls @ 619us

[17:10:49] GetGradient: 0.00355s, 64 calls @ 3550us

[17:10:49] PredictRaw: 0.009895s, 64 calls @ 9895us

[17:10:49] UpdateOneIter: 166.979s, 64 calls @ 166979414us

[17:10:49] ======== Monitor: GBTree ========
[17:10:49] BoostNewTrees: 166.964s, 64 calls @ 166964435us

[17:10:49] CommitModel: 5.2e-05s, 64 calls @ 52us

[17:10:49] ======== Device 0 Memory Allocations:  ========
[17:10:49] Peak memory usage: 1316MiB
[17:10:49] Number of alloca

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.1, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=20,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=64, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=3)

# Results

In [68]:
print(xgbLongModel.score(testInputs, testLabels[:,0]))
print(xgbLatModel.score(testInputs, testLabels[:,1]))

print(xgbLongModel.score(trainInputs, trainLabels[:,0]))
print(xgbLatModel.score(trainInputs, trainLabels[:,1]))

0.9130254662711935
0.9193616696595271
0.9974154443128466
0.9986935260096846


In [141]:
print(rfLongModel.score(testInputs, testLabels[:,0]))
print(rfLatModel.score(testInputs, testLabels[:,1]))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:    0.5s
[Para

In [76]:
print(rfLongModel.score(trainInputs, trainLabels[:,0]))
print(rfLatModel.score(trainInputs, trainLabels[:,1]))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:    0.8s
[Para

In [12]:
testInputs.shape

(228404, 130)

In [13]:
testDrive["samples"].shape

(926, 170)

In [69]:
print(xgbLongModel.score(testDrive["samples"], testDrive["labels"][:,0]))
print(xgbLatModel.score(testDrive["samples"], testDrive["labels"][:,1]))

-0.019343730623145428
-0.9437120361245339
